In [4]:
import torch, os
import pandas as pd
from transformers import pipeline, BertForSequenceClassification,BertTokenizerFast
from torch.utils.data import Dataset

In [5]:
from torch import cuda
device= 'cuda' if cuda.is_available() else 'cpu'
device

'cpu'

In [6]:
df=pd.read_csv(os.path.join("D:\ICTC2.0","train.csv"))
df.head()

,ben,guj,hin,kan,mal,ori,pan,tam,tel,urd,eng
0,sidny opera houser ashpashe thaka suijarlander...,sydney opera house aaspaasma yojayeli spardham...,sidney opera house kii prishthbhumi main switz...,sydney opera housein avarandalli aayojislagidd...,"sydney oppara hasiൻre parisarsliൽwech, switzeർ...",sidni opera houser pariveshtani madhyare anust...,sidney opera house de aale-duale sthit svissza...,sidney opera housein suzlil natant ant vilyatl...,sydney opera house parisarall jarigin i kridal...,"sidney opera house ke gurdonvah main kaayam, s...",Set in the surroundings of the Sydney Opera Ho...
1,con jivanu upar kaaj karbe sei anuyayi aigulo ...,sukshmajivanuona jooth par karaati asarne aadh...,jin sookshmaanuon ke samooh par ye kaam karti ...,ivugannu avu prenam biruv sookshmajivi gumpige...,badhikkunn sukshmajiviute waർggae atisthanmaki...,yeun bhootaanu goshthigudik upare eha kaam kar...,"inhan nuun anti- vistyle, anti- fungel, anti-p...",ews ent vaka nunnurigolin midhu vinapurikinden...,ivi prabhavan chupe sukshmajivas samuhaniki an...,inhen in jaraasam ke group ke lihaaz se jin pa...,"These are classified as anti-virals, anti-fung..."
2,bharter drut arth lenden vyavastha- upiai-ke a...,bharat bhugatan vyavasthao anaya adhikarakshet...,bharat kii sabase twarit bhugtan pranali - upi...,bharatad vegavad pavati vyavastheyad you.pi.i....,indiaille pettennu panamadaykanavunn nilevilul...,bharatar drut arth pradaan brivastha - eu.pi.a...,bharat de bhugtan karan wale shaylia da doosar...,indiavin yubis enepatum duridab panamseluttal ...,saguthunn bharat shighra chellimp vidhana - yu...,bhaarat ke adaegi ke nizaam ko digar dayera ak...,The possibility of linking India's payment sys...
3,fasal-poorvavarti evan fasal-paravarty caryakr...,paak poorveni ane paak pachhini pravrittio mat...,fasal kii kataai se pehle or uske baad die jan...,coylige munchin hagu coylen nantarad chatuvati...,villvetuppinumumpu villvetuppinusheavumulla pr...,amal-poorvavartti tatha amal-paravartti karyan...,fasal di kataai ton pihala ate bood vich us te...,aruvadiku mundaiy manllum bindaiy seyalbadugal...,"pantakotku mundu, taruvati panul koraku ichche...",kataai se kabel or kataai ke baad kii sargharm...,Loans for pre-harvest and post-harvest activit...
4,maldwar parikshar paddhati ekati drut paddhati...,gudamarg deetha thati tapas ek jhadpi padhdhat...,malashay-sambandhi pareekshan ek twarit vidhi ...,antah-gudnal pariksheyu kshipravad vidhanvagid...,computaർ samvidhana upyogitchull gudaparishodh...,malashai pariksha eka trit paddhati ate (jane ...,andakosh sambandi nirikhan ik jaladi on vaali ...,asana malakutal parichodanai woru viraivana va...,purishanalam dwara pariksh ock satvar vidhana ...,par-ractil maan ek tej tareeq car hai (ek taju...,The per-rectal examination is a quick method (...


In [7]:
l2id={}
id2l={}
for i,col in enumerate(df.columns):
    l2id[col]=i
    id2l[i]=col


In [8]:
tokenizer = BertTokenizerFast.from_pretrained("dbmdz/bert-base-turkish-uncased", max_length=512)

In [9]:
model = BertForSequenceClassification.from_pretrained("dbmdz/bert-base-turkish-uncased", num_labels=11, id2label=id2l, label2id=l2id)
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [10]:
size=df.shape
size

(1700, 11)

In [11]:
reshaped_data=pd.DataFrame(columns=['text','lang'])
for col in df.columns:
    values=df[col].tolist()
    tdf=pd.DataFrame({'text':values,'lang':[col]*len(values)})
    reshaped_data=reshaped_data.append(tdf,ignore_index=True)

reshaped_data.to_csv("reshaped_file.csv",index=False)

C:\Users\vinay\AppData\Local\Temp\ipykernel_18408\1701897921.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reshaped_data=reshaped_data.append(tdf,ignore_index=True)
C:\Users\vinay\AppData\Local\Temp\ipykernel_18408\1701897921.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reshaped_data=reshaped_data.append(tdf,ignore_index=True)
C:\Users\vinay\AppData\Local\Temp\ipykernel_18408\1701897921.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reshaped_data=reshaped_data.append(tdf,ignore_index=True)
C:\Users\vinay\AppData\Local\Temp\ipykernel_18408\1701897921.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reshaped_data=resh

In [12]:
df_org=pd.read_csv(os.path.join("D:\ICTC2.0",os.path.join("text_classification","reshaped_file.csv")))
df_org.head()


,text,lang
0,sidny opera houser ashpashe thaka suijarlander...,ben
1,con jivanu upar kaaj karbe sei anuyayi aigulo ...,ben
2,bharter drut arth lenden vyavastha- upiai-ke a...,ben
3,fasal-poorvavarti evan fasal-paravarty caryakr...,ben
4,maldwar parikshar paddhati ekati drut paddhati...,ben


In [13]:
df_org['label']=df_org.lang.map(lambda x: l2id[x.strip()])
df_org.head(-10)

,text,lang,label
0,sidny opera houser ashpashe thaka suijarlander...,ben,0
1,con jivanu upar kaaj karbe sei anuyayi aigulo ...,ben,0
2,bharter drut arth lenden vyavastha- upiai-ke a...,ben,0
3,fasal-poorvavarti evan fasal-paravarty caryakr...,ben,0
4,maldwar parikshar paddhati ekati drut paddhati...,ben,0
...,...,...,...
18685,How can we buy that?,eng,10
18686,It is made of chickpea flour and other herbs.,eng,10
18687,"An international cargo hub project, the Multi-...",eng,10
18688,Haha yeah!,eng,10


In [15]:
from sklearn.model_selection import train_test_split
x=df_org['text']
y=df_org['label']
X_train_temp, X_val_test, y_train_temp, y_val_test = train_test_split(x, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)

In [16]:
X_train_temp

12446                 hai devanga. naan nanjhak irukiheen.
11983                          its edaykak nataddukhirars?
12303    ind varusham rhodes thirumbhi vantirupatal itu...
11490     waah! tusin minuun irashaas vich paa ditaa hai I
10588    jiadatar ikakia vich tima banai gaia han jo ki...
                               ...                        
11284    bharat vich, beghatone cup ate aagaa khan tour...
11964             tabilufilyui netwarkil live stream achu.
5390                     mundin sal nodekke prayatnistini.
860      erapar amader royas fo bo evan arevati paschim...
15795    ye musanfin kii samaji zimm dari bin gai ki wo...
Name: text, Length: 14960, dtype: object

In [17]:
train_text=list(X_train_temp)
val_text=list(X_val)
test_text=list(X_test)

In [18]:
train_encodings = tokenizer(train_text, truncation=True, padding=True)
val_encodings  = tokenizer(val_text, truncation=True, padding=True)
test_encodings = tokenizer(test_text, truncation=True, padding=True)

In [19]:
class DataLoader(Dataset):
    """
    Custom Dataset class for handling tokenized text data and corresponding labels.
    Inherits from torch.utils.data.Dataset.
    """
    def __init__(self, encodings, labels):
        """
        Initializes the DataLoader class with encodings and labels.

        Args:
            encodings (dict): A dictionary containing tokenized input text data
                              (e.g., 'input_ids', 'token_type_ids', 'attention_mask').
            labels (list): A list of integer labels for the input text data.
        """
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        """
        Returns a dictionary containing tokenized data and the corresponding label for a given index.

        Args:
            idx (int): The index of the data item to retrieve.

        Returns:
            item (dict): A dictionary containing the tokenized data and the corresponding label.
        """
        # Retrieve tokenized data for the given index
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Add the label for the given index to the item dictionary
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        """
        Returns the number of data items in the dataset.

        Returns:
            (int): The number of data items in the dataset.
        """
        return len(self.labels)

In [20]:
train_label=list(y_train_temp)
test_label=list(y_test)
val_label=list(y_val)

In [21]:
train_dataloader = DataLoader(train_encodings, train_label)

val_dataloader = DataLoader(val_encodings, val_label)

test_dataset = DataLoader(test_encodings, test_label)

In [22]:
from transformers import TrainingArguments, Trainer

In [23]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    """
    Computes accuracy, F1, precision, and recall for a given set of predictions.
    
    Args:
        pred (obj): An object containing label_ids and predictions attributes.
            - label_ids (array-like): A 1D array of true class labels.
            - predictions (array-like): A 2D array where each row represents
              an observation, and each column represents the probability of 
              that observation belonging to a certain class.
              
    Returns:
        dict: A dictionary containing the following metrics:
            - Accuracy (float): The proportion of correctly classified instances.
            - F1 (float): The macro F1 score, which is the harmonic mean of precision
              and recall. Macro averaging calculates the metric independently for
              each class and then takes the average.
            - Precision (float): The macro precision, which is the number of true
              positives divided by the sum of true positives and false positives.
            - Recall (float): The macro recall, which is the number of true positives
              divided by the sum of true positives and false negatives.
    """
    # Extract true labels from the input object
    labels = pred.label_ids
    
    # Obtain predicted class labels by finding the column index with the maximum probability
    preds = pred.predictions.argmax(-1)
    
    # Compute macro precision, recall, and F1 score using sklearn's precision_recall_fscore_support function
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    
    # Calculate the accuracy score using sklearn's accuracy_score function
    acc = accuracy_score(labels, preds)
    
    # Return the computed metrics as a dictionary
    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }

In [24]:
training_args = TrainingArguments(
    # The output directory where the model predictions and checkpoints will be written
    output_dir='./TTC4900Model', 
    do_train=True,
    do_eval=True,
    #  The number of epochs, defaults to 3.0 
    num_train_epochs=3,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=32,
    # Number of steps used for a linear warmup
    warmup_steps=100,                
    weight_decay=0.01,
    logging_strategy='steps',
   # TensorBoard log directory                 
    logging_dir='./multi-class-logs',            
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps", 
    #fp16=True,
    load_best_model_at_end=True
)

In [25]:
trainer = Trainer(
    # the pre-trained model that will be fine-tuned 
    model=model,
     # training arguments that we defined above                        
    args=training_args,                 
    train_dataset=train_dataloader,         
    eval_dataset=val_dataloader,            
    compute_metrics= compute_metrics
)

In [38]:
trainer.train()


c:\Users\vinay\anaconda3\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: ERROR API key must be 40 characters long, yours was 14
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from you

  0%|          | 0/2805 [00:00<?, ?it/s]

{'loss': 2.3453, 'learning_rate': 2.5e-05, 'epoch': 0.05}


  0%|          | 0/59 [00:00<?, ?it/s]

c:\Users\vinay\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

{'eval_loss': 0.40628448128700256, 'eval_Accuracy': 0.8754010695187165, 'eval_F1': 0.8762657584729273, 'eval_Precision': 0.8825247315978206, 'eval_Recall': 0.8753395514055392, 'eval_runtime': 549.6497, 'eval_samples_per_second': 3.402, 'eval_steps_per_second': 0.107, 'epoch': 0.32}
{'loss': 0.4767, 'learning_rate': 4.537892791127542e-05, 'epoch': 0.37}


  0%|          | 0/59 [00:00<?, ?it/s]

{'eval_loss': 0.32796382904052734, 'eval_Accuracy': 0.8828877005347594, 'eval_F1': 0.8820594019088066, 'eval_Precision': 0.8934756928728768, 'eval_Recall': 0.884294808401028, 'eval_runtime': 549.3315, 'eval_samples_per_second': 3.404, 'eval_steps_per_second': 0.107, 'epoch': 0.37}
{'loss': 0.3889, 'learning_rate': 4.4454713493530505e-05, 'epoch': 0.43}


  0%|          | 0/59 [00:00<?, ?it/s]

{'eval_loss': 0.3433275520801544, 'eval_Accuracy': 0.8796791443850267, 'eval_F1': 0.8791865712967336, 'eval_Precision': 0.8918933768277149, 'eval_Recall': 0.8806164498864998, 'eval_runtime': 549.3603, 'eval_samples_per_second': 3.404, 'eval_steps_per_second': 0.107, 'epoch': 0.43}


In [ ]:
q=[trainer.evaluate(eval_dataset=df_org) for df_org in [train_dataloader, val_dataloader, test_dataset]]

pd.DataFrame(q, index=["train","val","test"]).iloc[:,:5]

In [ ]:
def predict(text):
    """
    Predicts the class label for a given input text

    Args:
        text (str): The input text for which the class label needs to be predicted.

    Returns:
        probs (torch.Tensor): Class probabilities for the input text.
        pred_label_idx (torch.Tensor): The index of the predicted class label.
        pred_label (str): The predicted class label.
    """
    # Tokenize the input text and move tensors to the GPU if available
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt").to("cuda")

    # Get model output (logits)
    outputs = model(**inputs)

    probs = outputs[0].softmax(1)
    """ Explanation outputs: The BERT model returns a tuple containing the output logits (and possibly other elements depending on the model configuration). In this case, the output logits are the first element in the tuple, which is why we access it using outputs[0].

    outputs[0]: This is a tensor containing the raw output logits for each class. The shape of the tensor is (batch_size, num_classes) where batch_size is the number of input samples (in this case, 1, as we are predicting for a single input text) and num_classes is the number of target classes.

    softmax(1): The softmax function is applied along dimension 1 (the class dimension) to convert the raw logits into class probabilities. Softmax normalizes the logits so that they sum to 1, making them interpretable as probabilities. """

    # Get the index of the class with the highest probability
    # argmax() finds the index of the maximum value in the tensor along a specified dimension.
    # By default, if no dimension is specified, it returns the index of the maximum value in the flattened tensor.
    pred_label_idx = probs.argmax()

    # Now map the predicted class index to the actual class label 
    # Since pred_label_idx is a tensor containing a single value (the predicted class index), 
    # the .item() method is used to extract the value as a scalar
    pred_label = model.config.id2label[pred_label_idx.item()]

    return probs, pred_label_idx, pred_label

In [ ]:
# Test with a an example text in Turkish
text = "Makine öğrenimi kendisi de daha da otomatik hale doğru ilerliyor."
# "Machine Learning itself is moving towards more and more automated"
predict(text)

In [ ]:
model_path = "text-classification-model"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)